## Converting a Retinanet (Keras) Model into Tensorflow Model + Make Predicitons


- Train Keras model
- Keras to single TensorFlow .pb file
- Load .pb file with TensorFlow and make prediction.
- (Optional) Visualize the graph in Jupyter notebook.

### Load Keras model from a file (optional)

In case you ran into the **incompatible with expected resource** issue with a model containing **BatchNormization** layers such as DenseNet, make sure to set the learning phase to 0 before loading the Keras model in a new session.
The error message looks like this when you call `tf.import_graph_def(graph_def)` later on during restoring. 
```
Conv_1/cond/ReadVariableOp/Switch was passed float from Conv_1/gamma:0 incompatible with expected resource
```

In [1]:
# show images inline
%matplotlib inline

from keras import backend as K
from show_graph import show_graph
from tensorflow.python.platform import gfile

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

# import miscellaneous modules
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import warnings

Using TensorFlow backend.


In [2]:
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

In [3]:
model = models.load_model('../../../models/output_tf1.h5', backbone_name='resnet50')







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








c:\users\ing cohen\anaconda3\envs\retinanet\lib\site-packages\keras\engine\saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
model.summary()

Model: "retinanet-bbox"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
_____________________________________________________________________________________

In [5]:
sess = K.get_session()
graph_def = sess.graph.as_graph_def()
# graph_def
#show_graph(graph_def)

### Check the input and output name
We will refer to them by name during TensorFlow inference.

In [6]:
# NOTE THAT ARE THREE VARIABLES!!!
print(model.outputs)

[<tf.Tensor 'filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, 300, 4) dtype=float32>, <tf.Tensor 'filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0' shape=(?, 300) dtype=int32>]


In [7]:
print(model.inputs)

[<tf.Tensor 'input_1:0' shape=(?, ?, ?, 3) dtype=float32>]


In [8]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [9]:
frozen_graph = freeze_session(K.get_session(), output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 306 variables.
INFO:tensorflow:Converted 306 variables to const ops.


Save `GraphDef` ProtoBuf as a binary file.

In [ ]:
tf.train.write_graph(frozen_graph, "model", "tf_model.pb", as_text=False)
tf.train.write_graph(frozen_graph, "model", "tf_model.pbtxt", as_text=True)
tf.train.export_meta_graph(filename='model/tf_model.meta')

## Load the pb file and inference
**Restart the Kernel first to clear previous training session.**

In [11]:
sess=tf.InteractiveSession()

In [12]:
f = gfile.FastGFile("./model/tf_model.pb", 'rb')
graph_def = tf.GraphDef()
# Parses a serialized binary message into the current message.
graph_def.ParseFromString(f.read())
f.close()

Instructions for updating:
Use tf.gfile.GFile.


In [13]:
sess.graph.as_default()
# Import a serialized TensorFlow `GraphDef` protocol buffer
# and place into the current default `Graph`.
tf.import_graph_def(graph_def)

### Locate the output Tensor by name

In [20]:
boxes_tf  = sess.graph.get_tensor_by_name('import/filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0')
scores_tf = sess.graph.get_tensor_by_name('import/filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0')
labels_tf = sess.graph.get_tensor_by_name('import/filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0')
predictions_tf = [boxes_tf, scores_tf, labels_tf]

### Prepare datasets for inference

In [21]:
# load image
image = read_image_bgr('../../test/images/badger.jpg')
# preprocess image for network
image = preprocess_image(image)
image, scale = resize_image(image)
# make as an array
image = np.expand_dims(image, axis=0)

In [22]:
image.shape

(1, 800, 800, 3)

### Make predictions - Compare Tensorflow Exported Model to Tensorflow Exported Model

###### Let's test the tensorflow predictions

In [23]:
predictions_tf = sess.run([boxes_tf, scores_tf, labels_tf], {'import/input_1:0': image})

In [24]:
predictions_tf

[array([[[ 78.79182, 166.07037, 773.99915, 707.87836],
         [ 78.40584, 163.49637, 777.69403, 712.1973 ],
         [295.378  , 172.32181, 776.48535, 546.73444],
         ...,
         [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
         [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
         [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32),
 array([[ 0.8806363 ,  0.2840796 ,  0.10478225,  0.07583722,  0.06363532,
          0.06175646,  0.05631873,  0.05437377,  0.05194205,  0.05049232,
         -1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        

In [26]:
predictions_tf[0].shape, predictions_tf[1].shape, predictions_tf[2].shape

((1, 300, 4), (1, 300), (1, 300))

In [27]:
boxes_tf = predictions_tf[0]
scores_tf = predictions_tf[1]
labels_tf = predictions_tf[2]

In [37]:
boxes_tf[0][:15]

array([[ 78.79182  , 166.07037  , 773.99915  , 707.87836  ],
       [ 78.40584  , 163.49637  , 777.69403  , 712.1973   ],
       [295.378    , 172.32181  , 776.48535  , 546.73444  ],
       [429.35175  , 209.2247   , 799.       , 768.20764  ],
       [161.6814   , 164.2366   , 686.212    , 419.6629   ],
       [220.21129  , 461.20728  , 649.1755   , 726.0253   ],
       [239.41685  , 204.50775  , 590.06287  , 639.5664   ],
       [ 33.357285 , 309.40683  , 543.657    , 710.9895   ],
       [248.1179   , 342.1353   , 751.85547  , 684.9498   ],
       [  2.0509644,  19.943928 , 774.1209   , 529.13293  ],
       [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
       [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
       [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
       [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
       [ -1.       ,  -1.       ,  -1.       ,  -1.       ]],
      dtype=float32)

In [38]:
scores_tf[0][:15]

array([ 0.8806363 ,  0.2840796 ,  0.10478225,  0.07583722,  0.06363532,
        0.06175646,  0.05631873,  0.05437377,  0.05194205,  0.05049232,
       -1.        , -1.        , -1.        , -1.        , -1.        ],
      dtype=float32)

In [39]:
labels_tf[0][:15]

array([ 1,  0,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1])

###### Let's test the retinanet predictions

In [29]:
model = models.load_model('../../../models/output_tf1.h5', backbone_name='resnet50')

In [30]:
predictions_retinanet = model.predict_on_batch(image)

In [31]:
predictions_retinanet

[array([[[ 78.79182, 166.07037, 773.99915, 707.87836],
         [ 78.40584, 163.49637, 777.69403, 712.1973 ],
         [295.378  , 172.32181, 776.48535, 546.73444],
         ...,
         [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
         [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
         [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32),
 array([[ 0.8806363 ,  0.2840796 ,  0.10478225,  0.07583722,  0.06363532,
          0.06175646,  0.05631873,  0.05437377,  0.05194205,  0.05049232,
         -1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        

In [32]:
predictions_retinanet[0].shape, predictions_retinanet[1].shape, predictions_retinanet[2].shape

((1, 300, 4), (1, 300), (1, 300))

In [33]:
boxes_retinanet = predictions_retinanet[0]
scores_retinanet = predictions_retinanet[1]
labels_retinanet = predictions_retinanet[2]

In [34]:
boxes_retinanet[0][:15]

array([[ 78.79182  , 166.07037  , 773.99915  , 707.87836  ],
       [ 78.40584  , 163.49637  , 777.69403  , 712.1973   ],
       [295.378    , 172.32181  , 776.48535  , 546.73444  ],
       [429.35175  , 209.2247   , 799.       , 768.20764  ],
       [161.6814   , 164.2366   , 686.212    , 419.6629   ],
       [220.21129  , 461.20728  , 649.1755   , 726.0253   ],
       [239.41685  , 204.50775  , 590.06287  , 639.5664   ],
       [ 33.357285 , 309.40683  , 543.657    , 710.9895   ],
       [248.1179   , 342.1353   , 751.85547  , 684.9498   ],
       [  2.0509644,  19.943928 , 774.1209   , 529.13293  ],
       [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
       [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
       [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
       [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
       [ -1.       ,  -1.       ,  -1.       ,  -1.       ]],
      dtype=float32)

In [35]:
scores_retinanet[0][:15]

array([ 0.8806363 ,  0.2840796 ,  0.10478225,  0.07583722,  0.06363532,
        0.06175646,  0.05631873,  0.05437377,  0.05194205,  0.05049232,
       -1.        , -1.        , -1.        , -1.        , -1.        ],
      dtype=float32)

In [36]:
labels_retinanet[0][:15]

array([ 1,  0,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1])

###### Compare the two arrays

In [40]:
np.array_equal(scores_retinanet, scores_tf)

True

In [41]:
np.array_equal(boxes_retinanet, boxes_tf)

True

In [42]:
np.array_equal(labels_retinanet, labels_tf)

True